# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Call csv file 
file = 'output_data/cities.csv'

cities_data = pd.read_csv(file)

cities_df = pd.DataFrame(cities_data)

cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bredasdorp,-34.53,20.04,64.40,68,100,4.70,ZA,1603655086
1,Wagar,16.15,36.20,88.86,32,32,11.74,SD,1603655086
2,Bira,49.00,132.47,34.07,88,0,3.56,RU,1603655086
3,Jamestown,42.10,-79.24,46.00,63,90,10.29,US,1603654935
4,Naze,28.37,129.48,64.40,88,90,3.36,JP,1603655071


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = cities_df["Humidity"]

In [7]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = cities_df.loc[(cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"] == 0) & \
                                   (cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,Qeshm,26.96,56.27,73.40,35,0,6.93,IR,1603655096
112,Salalah,17.02,54.09,77.00,57,0,5.82,OM,1603655112
171,Bilma,18.69,12.92,75.34,19,0,5.82,NE,1603655126
207,Keti Bandar,24.14,67.45,78.30,54,0,7.07,PK,1603655135
277,Koumac,-20.57,164.28,75.27,71,0,4.05,NC,1603655151
302,Buraidah,26.33,43.98,71.60,19,0,3.36,SA,1603655157
322,Umluj,25.02,37.27,79.61,58,0,5.55,SA,1603655225
333,Upington,-28.45,21.26,78.80,12,0,8.05,ZA,1603655227
358,Nkhotakota,-12.93,34.30,77.72,67,0,2.77,MW,1603655234
387,Agadez,19.75,10.25,73.90,20,0,8.23,NE,1603655240


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
49,Qeshm,IR,26.96,56.27,
112,Salalah,OM,17.02,54.09,
171,Bilma,NE,18.69,12.92,
207,Keti Bandar,PK,24.14,67.45,
277,Koumac,NC,-20.57,164.28,
302,Buraidah,SA,26.33,43.98,
322,Umluj,SA,25.02,37.27,
333,Upington,ZA,-28.45,21.26,
358,Nkhotakota,MW,-12.93,34.30,
387,Agadez,NE,19.75,10.25,


In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [18]:
for index, row in hotel_df.iterrows():
    params = {
    'location': f"{row['Lat']}, {row['Lng']}",
    'radius': 5000,
    'type': 'hotel',
    'key': g_key
    }
    
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except IndexError:
        hotel_df.loc[index, 'Hotel Name'] = np.nan

In [21]:
# Display the hotel dataframe
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != np.nan]
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
49,Qeshm,IR,26.96,56.27,Qeshm
112,Salalah,OM,17.02,54.09,Salalah
171,Bilma,NE,18.69,12.92,Bilma
207,Keti Bandar,PK,24.14,67.45,Keti Bandar
277,Koumac,NC,-20.57,164.28,Koumac


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))